### Streaming
스트리밍은 LLM 기반 애플리케이션의 응답성을 향상시키는데 매우 중요합니다. 완전한 응답이 준비되기 전에도 출력을 점진적으로 표시함으로써 스트리밍은 사용자 경험(UX)을 크게 향상시키며, 특히 LLM의 지연 시간을 처리할 때 더욱 그렇습니다.

- 에이전트 진행 상황과 함께 스크리밍 하려면 
`stream()`, 또는 `astream()` 메소드 사용, `stream_mode="update"`로 하면 이이전트의 모든 단계가 끝날 때마다 이벤트가 발생.

In [1]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[get_weather],
)
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: agent
content: [{'type': 'tool_call', 'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_ouzVPlu0WHuAWk71mzQUzH8C'}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]
step: agent
content: [{'type': 'text', 'text': "San Francisco weather: It's always sunny in San Francisco!\n\nIf you’d like exact details (temperature, wind, humidity, hourly forecast), I can pull those up."}]


- LLM에서 생성된 토큰을 스트리밍 하려면 `stream_mode="message"로 설정합니다.

In [2]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[get_weather],
)
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: agent
content: [{'type': 'tool_call_chunk', 'id': 'call_dGegzHKCgwyrMz7aD8zq6tDj', 'name': 'get_weather', 'args': '', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '{"', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'city', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '":"', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'San', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': ' Francisco', 'index': 0}]


node: agent
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '"}', 'index': 0}]


node: agent
content: []


node: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]


node: agent
content: []


node: agent
content: [{'type': 'text', 'text': 'It'}]


node: agent
content: [

- 도구가 실행되면서 업데이트를 스트리밍하려면 `get_stream_writer`를 사용.

In [4]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

Looking up data for city: San Francisco
Acquired data for city: San Francisco


- `stream_mode=["updates", "custom"]` 스트림 모드를 목록으로 전달하여 여러 모드를 지원할 수 있습니다.

In [5]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[get_weather],
)

for stream_mode, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]
):
    print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")

stream_mode: updates
content: {'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 216, 'prompt_tokens': 132, 'total_tokens': 348, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CGyEa8AKXAieMh0UrMu5q8pib9wM7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d6f712d4-8f76-469e-8fd1-ba7fb9a307b6-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_MboQ06dAgPa1cae2q64lrZtf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens': 216, 'total_tokens': 348, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0